### Import Libraries

In [20]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras import layers, models
from tensorflow.keras.utils import plot_model

### Loading the data
The PKLot dataset contains 12,416 images of parking lots extracted from surveilance camera frames. There are images on sunny, cloudy, and rainy days and the parking spaces are labeled as occupied or empty. We have converted the original annotations to a variety of standard object detection formats by enclosing a bounding box around the original dataset's rotated rectangle annotations.

In [8]:
# Load and preprocess the PKLot dataset
data_dir = 'train/'
annotations_file = 'train/_annotations.csv'
test_data_dir = 'test/'
test_annotations_file = 'test/_annotations.csv'
image_size = (224, 224)

In [9]:
annotations = pd.read_csv(annotations_file)
test_annotations = pd.read_csv(test_annotations_file)

### Data Preparation

In [7]:
# Creating arrays for images and labels depicting if the space is occupied or empty
images = []
labels = []

for image_file in annotations['filename'].unique():
    image_path = os.path.join(data_dir, image_file)
    image = Image.open(image_path).resize(image_size)
    images.append(np.array(image))
    
    # Check if any annotation for 'space-occupied' exists in the image annotations
    if annotations[(annotations['filename'] == image_file) & (annotations['class'] == 'space-occupied')].shape[0] > 0:
        labels.append(1)
    else:
        labels.append(0)
# Converting the arrays into numpy arrays
images = np.array(images)
labels = np.array(labels)

In [10]:
# Creating arrays for images and labels depicting if the space is occupied or empty
test_images = []
test_labels = []

for image_file in test_annotations['filename'].unique():
    image_path = os.path.join(test_data_dir, image_file)
    image = Image.open(image_path).resize(image_size)
    test_images.append(np.array(image))
    
    # Check if any annotation for 'space-occupied' exists in the image annotations
    if test_annotations[(test_annotations['filename'] == image_file) & (test_annotations['class'] == 'space-occupied')].shape[0] > 0:
        test_labels.append(1)
    else:
        test_labels.append(0)

# Converting the arrays into numpy arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)

### Modeling

#### Defining the model

In [12]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification output

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

#### Training and evaluating the model

In [14]:
# Train the model
history = model.fit(images, labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))

# Evaluate the model
y_pred = model.predict(test_images)
y_pred_binary = (y_pred > 0.5).astype(int)

val_accuracy = accuracy_score(test_labels, y_pred_binary)
val_classification_rep = classification_report(test_labels, y_pred_binary, target_names=['Empty', 'Occupied'])

print(f'Validation Accuracy: {val_accuracy:.2f}')
print('Validation Classification Report:\n', val_classification_rep)

Epoch 1/10
266/266 [==============================] - 462s 2s/step - loss: 10.5101 - accuracy: 0.9076 - val_loss: 0.0645 - val_accuracy: 0.9762
Epoch 2/10
266/266 [==============================] - 465s 2s/step - loss: 0.5771 - accuracy: 0.8576 - val_loss: 0.2069 - val_accuracy: 0.8939
Epoch 3/10
266/266 [==============================] - 470s 2s/step - loss: 0.2787 - accuracy: 0.8713 - val_loss: 0.3919 - val_accuracy: 0.8199
Epoch 4/10
266/266 [==============================] - 467s 2s/step - loss: 0.2805 - accuracy: 0.8667 - val_loss: 0.2256 - val_accuracy: 0.8791
Epoch 5/10
266/266 [==============================] - 467s 2s/step - loss: 0.1552 - accuracy: 0.9197 - val_loss: 0.0811 - val_accuracy: 0.9720
Epoch 6/10
266/266 [==============================] - 466s 2s/step - loss: 0.2454 - accuracy: 0.8740 - val_loss: 0.2659 - val_accuracy: 0.8914
Epoch 7/10
266/266 [==============================] - 466s 2s/step - loss: 0.2336 - accuracy: 0.8866 - val_loss: 0.0947 - val_accuracy: 0.964

We have near perfect results for the dataset